In [1]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")
X = {}
y = {}
X["train"] = np.load("./train_feature_save.npy",allow_pickle=True )
y["train"] = np.load("./train_label_save.npy").squeeze()
X["dev"] = np.load("./test_feature_save.npy")
y["dev"] = np.load("./test_label_save.npy").squeeze()
X_text_train = np.load("/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/LR/IF/Logistic_speech/X_train_text.npy")
X_text_dev = np.load("/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/LR/IF/Logistic_speech/X_test_text.npy")
X["train"].shape, X["dev"].shape, y["dev"].shape

((9632, 768), (1071, 768), (1071,))

In [2]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=0.1)
model.fit(X["train"], y["train"])
model.score(X["dev"], y["dev"])

0.8935574229691877

In [3]:
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]
gradient_train.shape, gradient_dev.shape

((9632, 769), (1071, 769))

In [4]:
#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + 10 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inverse_hessian = np.linalg.inv(hessian)

In [5]:
eps = 1 / X["train"].shape[0]
delta_k = -eps * inverse_hessian @ gradient_train.T
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k
delta_pred.shape

(1071, 9632)

In [6]:
ls

CSV_essay_bertlin_alg1.ipynb  old_predictions.npy
Smallest_k.ipynb              run_retrain.sh
Smallest_k.py                 run_tagger.sh
Use_approK.ipynb              save_feature.py
__pycache__/                  save_feature.sh
appro_ks_IP.npy               test_feature_save.npy
bert_util.py                  test_label_save.npy
essay_data/                   train_feature_save.npy
model_save_feature/           train_label_save.npy
new_predictions.npy


In [7]:
app_k = np.array(np.load("appro_ks_IP.npy", allow_pickle=True).squeeze())
pred = np.load("old_predictions.npy", allow_pickle=True)
app_k_nonon=[]
for i in range(X["dev"].shape[0]):
    print(app_k[i])
    if app_k[i] == None:
        app_k_nonon.append(X["train"].shape[0])
    else:
        app_k_nonon.append(app_k[i])

64
None
None
301
421
257
24
None
5
None
None
2120
15
306
None
196
93
2726
532
None
None
None
45
56
42
None
None
3
None
27
877
1112
None
None
None
17
3616
976
None
62
None
None
188
None
1175
None
None
1086
48
415
None
15
879
None
None
None
None
369
None
5
None
2872
None
None
None
None
None
None
None
366
191
99
None
121
None
423
106
None
None
None
83
None
56
None
None
None
497
2525
1316
None
86
284
None
1
None
None
None
None
None
None
None
34
541
2098
5
None
304
70
12
None
None
None
None
16
None
3
None
None
None
None
424
28
None
81
None
30
None
None
None
None
None
None
None
113
None
127
2167
None
None
None
528
81
133
None
None
87
None
13
None
None
None
38
89
None
None
92
78
None
96
None
961
36
None
None
None
None
None
28
5
None
1
None
None
None
None
439
None
98
None
None
15
None
33
90
None
1627
None
None
None
543
25
55
None
None
None
None
66
83
279
None
None
82
39
None
217
None
None
None
None
None
None
36
None
None
150
1
None
25
2
None
1550
None
2392
None
1621
None
None
4621
None
31
1
49

In [8]:
X["train"].shape[0]

9632

In [9]:
sort_dev_k = np.array(app_k_nonon, dtype=int).argsort()
sort_dev_k

array([ 93, 170, 215, ..., 717, 371,  81])

In [10]:
app_k_nonon[253]

9632

In [11]:
header = ["test_movie_id", "test_movie_review_text", "old_prediction", "right_prediction", "k", "successful_flip"]
for i in range(X["train"].shape[0]):
    header.append("train_example_" + str(i+1))

In [12]:
new_predictions = np.load("new_predictions.npy", allow_pickle=True)

In [13]:
data = []
for test_movie_id in sort_dev_k:
    
    test_point = []
    # id
    test_point.append(test_movie_id)
    # test_movie_review_text
    test_point.append("label: " + str(y["dev"][test_movie_id]) + "  doc: " + str(X_text_dev[test_movie_id]))
    test_point.append(pred[test_movie_id].item())
    test_point.append(np.round(pred[test_movie_id].item()) == y["dev"][test_movie_id].item())
    # k
    k = app_k[test_movie_id]
    test_point.append(str(k))
    print("k", k)
    if k == None:
        test_point.append(None)
        data.append(test_point)
        continue
    k = int(k)
    test_point.append(np.round(new_predictions[test_movie_id]) != np.round(pred[test_movie_id].item()))
        
    
    if pred[test_movie_id] > 0.5:
        top_k_index = delta_pred[test_movie_id].argsort()[-k:]
    else:
        top_k_index = delta_pred[test_movie_id].argsort()[:k]
    
    # training examples
    for idx in top_k_index:
        test_point.append("id: " + str(idx) + "  label: " + str(y["train"][idx]) + "  doc: " + str(X_text_train[idx]))
    data.append(test_point)

k 1
k 1
k 1
k 1
k 1
k 1
k 1
k 1
k 2
k 2
k 2
k 3
k 3
k 3
k 3
k 3
k 3
k 3
k 3
k 3
k 3
k 4
k 4
k 4
k 4
k 4
k 5
k 5
k 5
k 5
k 5
k 5
k 5
k 5
k 5
k 5
k 5
k 6
k 6
k 6
k 6
k 6
k 7
k 7
k 7
k 7
k 8
k 8
k 9
k 9
k 9
k 10
k 10
k 10
k 10
k 10
k 11
k 11
k 11
k 11
k 11
k 12
k 12
k 12
k 12
k 12
k 12
k 13
k 13
k 13
k 13
k 14
k 14
k 15
k 15
k 15
k 15
k 15
k 16
k 16
k 16
k 17
k 17
k 18
k 18
k 18
k 18
k 19
k 19
k 20
k 20
k 20
k 21
k 21
k 22
k 22
k 22
k 22
k 22
k 22
k 23
k 23
k 23
k 24
k 24
k 24
k 25
k 25
k 25
k 25
k 25
k 25
k 25
k 27
k 27
k 27
k 27
k 27
k 28
k 28
k 30
k 30
k 31
k 31
k 32
k 32
k 32
k 33
k 34
k 34
k 34
k 34
k 34
k 34
k 36
k 36
k 36
k 36
k 37
k 37
k 38
k 38
k 38
k 39
k 39
k 40
k 41
k 42
k 42
k 42
k 42
k 42
k 43
k 43
k 45
k 45
k 46
k 46
k 47
k 48
k 48
k 49
k 52
k 54
k 54
k 54
k 55
k 55
k 55
k 55
k 56
k 56
k 56
k 56
k 57
k 57
k 57
k 58
k 58
k 58
k 58
k 60
k 61
k 62
k 63
k 63
k 63
k 63
k 63
k 64
k 64
k 64
k 66
k 66
k 68
k 68
k 69
k 69
k 70
k 70
k 70
k 71
k 71
k 71
k 71
k 73
k 74
k 74
k 75
k 76
k

In [14]:
import csv

with open('CSV_speech_bertlin_alg1.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)


In [15]:
ls

CSV_essay_bertlin_alg1.ipynb  new_predictions.npy
CSV_speech_bertlin_alg1.csv   old_predictions.npy
Smallest_k.ipynb              run_retrain.sh
Smallest_k.py                 run_tagger.sh
Use_approK.ipynb              save_feature.py
__pycache__/                  save_feature.sh
appro_ks_IP.npy               test_feature_save.npy
bert_util.py                  test_label_save.npy
essay_data/                   train_feature_save.npy
model_save_feature/           train_label_save.npy


In [16]:
pwd

'/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/BERT/Bert_SST/Bert_linear_speech'